In [1]:
#%env CUDA_VISIBLE_DEVICES=0

import sys
sys.path.append('./modules/multi_summ')

In [2]:
import torch
device = torch.device("cuda")
torch.randn([1, 2]).to(device)

tensor([[-0.4738, -0.3302]], device='cuda:0')

# 1. Scouter

In [3]:
%%time
from modules.scouter_handler import ScouterHandler
from modules.scouter.config import default_addr, default_newspaper_idx, default_timeline_idx
from modules.topic_generator import TopicGenerator

CPU times: user 2.13 s, sys: 384 ms, total: 2.52 s
Wall time: 2.27 s


In [4]:
keyword = "삼성전자"
out_path = "results/"

scouter = ScouterHandler(addr=default_addr, size=1000)

In [21]:
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'postingDate', 'newsTitle', 'extContent', 'postingDate', 'analyzed_text'])
doc_info_list = scouter.search(query_body, data_type=default_newspaper_idx, max_num_doc=1000, trim_lower=False)

Query : 삼성전자


[2019-11-21 07:20:57,055 INFO] GET http://155.230.34.145:9200/newspaper/_search?scroll=10m&size=1000 [status:200 request:6.387s]


Scroll idx : 1 (1000 docs)
Total retrieved Doc # :  1000



In [5]:
doc_info_list[0]['extContent']

'세계 최대 자산운용사인 블랙록이 삼성전자 지분 5.03%를 보유하고 있다고 공시했다. 블랙록은 지난 1월에만 삼성전자 지분 950억원어치를 매입하며 지분율을 확대했다. 7일 블랙록은 지난달 장내매수로 삼성전자 지분을 5%  이상 보유하게 됐다고 공시했다. 지난 1월에 삼성전자 주식 약 211만주를 순매수했고, 매수단가를 기준으로 한 취득 규모는 약 950억원이다. 지난달 블랙록은 삼성전자 주식을 한 주당 4만5000원대에서 집중 매입했다. 이날 삼성전자 주가는 전 거래일보다 150원(0.32%) 하락한 4만6200원으로 장을 마쳤다.'

In [9]:
date = '2019-05-08'
topic_gen = TopicGenerator(keyword, out_path)
topic_doc = topic_gen.process_data(date)
clusters = topic_doc['clusters']

for i in range(len(clusters)):
    print('Topic {} keywords :'.format(i+1), clusters[i]['keywords'])
    print('Topic {} news article & # of doc {}:'.format(i+1, len(clusters[i]['news_source'])))
    for doc in clusters[i]['news_source']:
        print(doc['newsTitle'])
    print()

for i in range(len(clusters)):
    keyword_search_test = scouter.search_for_topic(clusters[i]['keywords'])
    print('Topic ', i)
    for doc in keyword_search_test: print(doc['newsTitle'])

[2019-11-21 07:18:31,657 INFO] GET http://155.230.34.145:9200/keyword_news/_search [status:200 request:0.018s]
[2019-11-21 07:18:31,697 INFO] GET http://155.230.34.145:9200/newspaper/_search [status:200 request:0.034s]
[2019-11-21 07:18:31,788 INFO] GET http://155.230.34.145:9200/newspaper/_search [status:200 request:0.083s]


Topic 1 keywords : ['수출', '부진', '경상흑자']
Topic 1 news article & # of doc 9:
불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저
불안한 경상흑자  1분기 6년9개월만에 최소...4월 적자 가능성(종합)
수출 부진  쪼그라든 경상흑자 4월 적자 전환하나?
수출 부진에 경상수지  휘청  6년9개월만 흑자 최소(종합)
수출 부진  1분기 경상흑자 112.5억달러...6년9개월만에  최소
1분기 경상수지 6년9개월 만에 최저 수출 부진 속  불황형 흑자
1분기 경상수지 흑자 112.5억달러 2012년 이후 최소
경상수지 83개월 흑자행진에도...규모는 7년 만에 최소
경상수지 83개월 연속 흑자 끊기나 외국인 배당금이 좌우

Topic 2 keywords : ['트럼프', '시진핑', '관세']
Topic 2 news article & # of doc 15:
트럼프  추가 관세  위협 속 미중 9∼10일 무역협상 나선다
뉴욕증시, 미중 무역협상 우려 다시 커지며 급락 다우 1.79%
무역전쟁 우려에 美증시   관세 위협, 위험으로 인식
中언론  美과 무역협상 어려움 대처할 자신있다
채권왕  건들락, 美 관세 인상 가능성 50%
美 3250억弗 대중 추가관세 실행 불가능
中 4월 수출 전년比 2.7%  대미 무역도 급감
미 중 마지막 단두대 협상 류허의 입에 달렸다
미중 무역전쟁 우려, 亞증시 줄줄이 하락
트럼프 블러핑 김정은에겐 통했는데 시진핑은?
엄포 아니었다 美  10일 중국산 관세 10% 25%로 인상
中 트럼프 협박에도 협상 계속하는 진짜 이유는?
美 中  무역전쟁  위기 뉴욕증시 급락, 코스피 꿈틀
트럼프 관세  카운트다운  美월가  최악 대비하라  경고
한번 해보자  중국도 강경파들 급속히 입지 넓혀

Topic 3 keywords : ['EU', '유로존', '하락']
Topic 3 news article & # of doc 8:
EU, 유로존 GDP 성장률 하향 조정 1.3% 1.2%(종합)


[2019-11-21 07:18:31,974 INFO] GET http://155.230.34.145:9200/newspaper/_search [status:200 request:0.131s]
[2019-11-21 07:18:32,035 INFO] GET http://155.230.34.145:9200/newspaper/_search [status:200 request:0.023s]
[2019-11-21 07:18:32,167 INFO] GET http://155.230.34.145:9200/newspaper/_search [status:200 request:0.117s]


Topic  2
주요지수, 미 중 무역협상 차질 가능성에 투자심리 위축  다우0.87%
원/달러 브렉시트 불확실성에 하락폭 제한-키움
유럽경제  먹구름  글로벌IB, 성장 전망 속속 낮춰
여전한 악재, 유럽發 불확실성
코스피, 외국인 자금유입 주춤  속도조절하며 유입 지속될 것
경기 불확실성 커지는 유럽 동향 모니터링 필요
유럽 골칫덩이 伊 포퓰리즘 남발에 성장률 1% 0.2%
미 중 불확실성에 뉴욕증시 하락 국내 하락폭은 크지 않을 듯
믿었던 독일마저  휘청   유로존 경제 통째로 흔들리나
유럽쇼크에 强달러 외국인 나흘째  팔자
Topic  3
Topic  4
안갯속 브렉시트 英의회  플랜B  준비 압박
융커 EU집행위원장  브렉시트, 낙관적이지 않아 노딜 배제 못해
미 중 무역전쟁에 불안한 증시..美 다우지수 0.9% 하락
재앙이 된 브렉시트  英 은행자산 1조파운드 빠져나간다
브렉시트 앞두고 유럽 방문 나선 윤석헌 김용범
의회 오락가락에 英기업들 분통  떠나든 남든 빨리 정하라
미중 무역전쟁 막내리나  연장 Vs 결렬  갈림길
브렉시트 코앞인데 무역협상 지지부진  영국 교역  벼랑끝
美 中 EU  3대 경제권  격랑 한국 경제도 후폭풍 촉각


In [ ]:
query_body = ScouterHandler.make_count_query_body(keyword, from_date="2019-05-01", to_date="2019-12-31")
trend_data = scouter.search_for_trend(query_body, max_num_doc=100)

In [ ]:
from collections import Counter
Counter({d['key_as_string']:d['doc_count'] for d in trend_data['aggregations']['date_range']['buckets'][0]['group_by_date']['buckets']})

# 2. Multi-document Summary

In [7]:
%%time
from modules.multi_doc_summary import MultiDocSummary

multi_doc_summarizer = MultiDocSummary('0', '0')

CPU times: user 2.13 s, sys: 244 ms, total: 2.37 s
Wall time: 2.24 s


In [11]:
%%time
multi_doc_summarizer.process_data([doc['extContent'] for doc in clusters[0]['news_source']][:8])

CPU times: user 3.15 s, sys: 976 ms, total: 4.12 s
Wall time: 4.12 s


'1분기 경상수지 흑자 규모가 6년9개월래 최소폭 축소됐다. 전년동기 대비 9.4% 줄었다. 전년 동월비 9.4% 감소한 수치다. 1분기(4분기) 이후 가장 저조한 성적을 보였기 때문으로 풀이된다. 1분기 경제성장률은 전분기보다 9.7% 감소했다. 전년. . . . . . . . '

# 3. TimelineSummary

In [12]:
from modules.timeline_summary import TimelineSummary

In [13]:
keyword = '미국 중국 무역'
outpath = './results/'

In [14]:
%%time
scouter = ScouterHandler(addr=default_addr, size=1000)
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'postingDate', 'extContent', 'extract'])
doc_info_list = scouter.search(query_body, data_type=default_timeline_idx, max_num_doc=200, trim_lower=False)

Query : 미국 중국 무역


[2019-11-21 07:19:06,660 INFO] GET http://155.230.34.145:9200/summary_by_time/_search?scroll=10m&size=200 [status:200 request:0.914s]


Scroll idx : 1 (200 docs)
Total retrieved Doc # :  200

CPU times: user 360 ms, sys: 40 ms, total: 400 ms
Wall time: 1.26 s


In [15]:
%%time
timeline_summ = TimelineSummary(keyword, outpath)
summary, _ = timeline_summ.process_data(doc_info_list)
dates = [sentence['date'] for sentence in summary]
timeline_sen = [' '.join(sentence['compResult']) for sentence in summary]
normal_sentence = [sentence['article'][0]['sentence'].strip() for sentence in summary]
score = [sentence['article'][2]['score'] for sentence in summary]

A :  0.05431962013244629
B :  0.18769121170043945
C :  0.009619712829589844
C-sub :  0.02051830291748047
 >> phase 4. calculate next nodes
timeline
['2017-09-20', '2018-11-15', '2018-11-29', '2019-01-08', '2019-05-07', '2019-05-11']
date_idx: 1/6
date_idx: 2/6
date_idx: 3/6
date_idx: 4/6
date_idx: 5/6
 >> phase 4. Make beamsearch file
D :  0.004110813140869141
E :  2.922905206680298
CPU times: user 444 ms, sys: 364 ms, total: 808 ms
Wall time: 3.2 s


In [ ]:
list(zip(dates, normal_sentence))

In [ ]:
list(zip(dates, timeline_sen))

# 4. Image Selection

In [3]:
from modules.image_selector import ImageSelectionModule

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [4]:
%%time
imageSelector = ImageSelectionModule("aa")




INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


CPU times: user 5.13 s, sys: 336 ms, total: 5.47 s
Wall time: 5.44 s


In [8]:
%%time
imageSelector.process_data("당장 이란 군부는 이날 미국이 이란산 원유수출에 대한 제한적 제재유예를 연장하지 않은 것과 관련해 워너유 수송로인 호르무츠해협을 봉쇄하겠다고 경고하며 맞대응했다.", "./a.jpg", download_limit=50)

image and caption downloaded
image and caption downloaded
image and caption downloaded
image and caption downloaded
image and caption downloaded


KeyboardInterrupt: 

In [5]:
import re

date_list = ["2019-05-07", "2019-05-08", "2019-05-09", "2019-03-28"]
#for date in date_list:
#    os.mkdir("./results/"+date)

result_list = [
    [
        "트럼프의 뒤집기 전술에 딜레마 빠진 中 협상 해도 문제",
        "韓외환보유액 순위 7개월만에 9위로 밀려 4월말 12억달러",
        "엠플러스, 70억 2차전지 제조설비 공급.. 해외수주 증가",
        "하나은행-주금공, 한부모가족 전세대출 요건완화 금리인하",
        "관세전쟁 악몽 재현  코스피 2170선붕괴"
    ],
    [
        "불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저",
        "트럼프 추가 관세 위협 속 미중 9∼10일 무역협상 나선다",
        "EU, 유로존 GDP 성장률 하향 조정 1.3% 1.2%(종합)",
        "개인도 주택금융공사 발행 MBS 투자 가능해진다",
        "글로벌 기업 CFO 35.6%  브렉시트 시한 또 연장될 것"
    ],
    [
        "끝나지 않은 무역전쟁 악몽",
        "중고차 대출한도, 차값의 110% 이내로 제한된다",
        "비씨카드 이제 중국에서 모바일로 결제하세요",
        "한은, 새로운 금융상황지수 도입... 2017년 4분기 이후 완화기조 지속",
        "삼성자산운용, 개인연금 설정액 1조 돌파"
    ],
    [
        "선거 임박 터키 리라 방어 임시방편에 금융시장 요동",
        "전세 월세 내준 임대가구 금융부채 372조원",
        "조양호 대표이사 박탈 영향은 미미할 것",
        "트럼프, OPEC에 유가 상승 경고  OPEC 진정해야",
        "상장법인 재감사 비율 증가세 보수는 2.6배 높아"
    ]
]
for date, sen_list in zip(date_list, result_list):
    print("[*] DATE : " + date)
    for i, sen in enumerate(sen_list):
        sen = re.sub("\s+", " ", sen)
        print(i, sen)
        imageSelector.process_data(sen, "./results/{}/daily_{}.jpg".format(date, i))

[*] DATE : 2019-05-07
0 트럼프의 뒤집기 전술에 딜레마 빠진 中 협상 해도 문제
10.515323400497437
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']
(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : graph
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : others
10.jpg : , Predict : graph
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predi

0.19605398178100586
recommended image : 30.jpg
3.9715960025787354
[*] DATE : 2019-05-08
0 불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저
11.173689603805542
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']
(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , P

vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : others
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : others
12.jpg : , Predict : graph
13.jpg : , Predict : others
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : others
34.jpg : , Predict : others
35.jpg : , Predict : others
36.jpg : , Predict : othe

0.1919093132019043
recommended image : 24.jpg
4.003591299057007
4 삼성자산운용, 개인연금 설정액 1조 돌파
10.96693205833435
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']
(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : graph
2.jpg : , Predict : others
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict

vgg predict - success
1.jpg : , Predict : others
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : others
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : others
10.jpg : , Predict : graph
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : graph
14.jpg : , Predict : others
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : others
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : graph
32.jpg : , Predict : others
33.jpg : , Predict : others
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : grap

In [10]:
result_list = [
    [
        "지난해(181억7000만달러)에 비해 증가 폭이 줄었지만 한국 외환보유액은 10년간 증가세를 보이며 두 배로 확대됐다.",
        "지난달 외환보유액이 4,055억 달러로 사상 최대 기록을 다시 갈아치웠다.",
        "외환보유액 1위 중국 3만879억달러, 2위는 일본 1만2793억달러 우리나라의 외환보유액 규모는 2월말 기준 소폭 감소하며 세계 8위 수준(1월)을 유지한 것으로 나타났다.",
        "우리나라의 외환보유액은 지난해 8월말 4011억달러을 기록해 국가별 순위에서 인도(4001억달러)를 제치고 9위에서 8위로 올라선 바 있다."
    ],
    [
        "15일(현지시간) 영국 하원에서 브렉시트 합의안이 큰 표 차로 부결되며 불확실성이 커진 점이 원/달러 환율 상승 배경인 것으로 풀이된다.",
        "테리사 메이 영국 총리는 표결 직후 성명을 통해 예고한 대로 13일 노딜 브렉시트 여부를 하원 표결에 부치겠다고 밝혔다.",
        "21~22일 EU정상회담 메이 빈손회담 가능성 커져 유럽연합(EU)이 브렉시트(영국의 EU 탈퇴) 시기 연기해달라는 영국의 요청에 영국 의회가 합의문을 승인해야 한다고 밝혔다."
    ],
    [
        "도널드 트럼프 미국 대통령이 오는 3월 1일까지였던 중국과의 무역 협상 시한을 연장하기로 했다.",
        "겅솽 대변인의 발표를 앞두고 중국 상무부도 류허 중국 부총리가 오는 9~10일 예정대로 미국에서 협상을 벌일 것이라고 성명을 내기도 했다.",
        "지난해 8월 22~23일 워싱턴DC에서는 왕서우원 중국 상무부 부부장과 데이비드 멀패스 미국 재무부 차관이 이끈 양국 협상 대표단이 무역협상을 진행했다.",
        "중국 정부는 미국의 관세 폭탄 에 대응해 6월 1일부터 600억 달러 규모의 미국산 제품에 대해 5∼25%의 보복 관세를 부과하기로 했다.",
        "미국 정부는 23일(현지시간) 자국 통화를 절하하는 국가들에 상계관세를 부과하는 규정을 추진한다고 밝혔다.",
        "미국 재무부는 28일(현지시간) 한국과 중국, 일본을 포함한 9개국을 환율 관찰대상국으로 지정, 발표했다."
    ],
    [
        "하지만 14개 산유국으로 구성된 OPEC과 러시아 등이 가격 정상화를 위해 올해부터 하루 120만 배럴에 해당하는 대규모 감산을 결정했다.",
        "서부텍사스산원유(WTI)는 2일 전 거래일 대비 2.25% 오른 배럴당 46.54달러에 장을 마쳤으며 북해산 브렌트유는 같은날 장중 5.1% 가까이 뛰기도 했다.",
        "25일(현지시간) 트럼프 대통령은 자신의 트위터 계정을 통해 유가가 너무 높다며 OPEC은 제발 진정하라고 촉구했다.",
        "전날 OPEC(석유수출국기구)의 감산을 비판한 도널드 트럼프 미국 대통령의 트윗에도 불구하고 OPEC이 감산을 계속하겠다고 밝히면서다.",
        "당장 이란 군부는 이날 미국이 이란산 원유수출에 대한 제한적 제재유예를 연장하지 않은 것과 관련해 워너유 수송로인 호르무츠해협을 봉쇄하겠다고 경고하며 맞대응했다.",
        "미국 내 가솔린 소비자가는 이날 현재 갤런당 2.84달러에 달해 미국 소비자들의 심리적 저항선인 3달러에 근접하고 있다."
    ]
]

for date, sen_list in zip(date_list[-1:], result_list[-1:]):
    print("[*] DATE : " + date)
    for i, sen in enumerate(sen_list[-1:], start=len(sen_list)-1):
        sen = re.sub("\s+", " ", sen)
        print(i, sen)
        imageSelector.process_data(sen, "./results/{}/timeline_{}.jpg".format(date, i))

[*] DATE : 2019-03-28
5 미국 내 가솔린 소비자가는 이날 현재 갤런당 2.84달러에 달해 미국 소비자들의 심리적 저항선인 3달러에 근접하고 있다.
3.62126088142395
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
(9, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : others
0.05882668495178223
recommended image : 5.jpg
3.847708225250244


# 5. Outlook

In [22]:
%%time

from modules.outlook import ForecastSentence

keyword = '삼성전자'
outpath = 'result/'
today = '2017-06-11'
forecast = ForecastSentence(keyword, outpath)

sentences = forecast.process_data(doc_info_list, today)
sentences[0:10]

/workspace/wise_reporter/modules/forecast/prediction.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = a(output)


CPU times: user 1.42 s, sys: 8 ms, total: 1.43 s
Wall time: 791 ms


[' 삼성전자의 올 2분기 실적이 매출액 58조9000억원, 영업이익 14조500억원으로 전년 동기에 비해 각각 15.7%, 72.5% 증가할 것으로 전망, 특히 영업이익이 시장의 기대치 12조9000억원을 상회할 것이라고 내다봤다.',
 ' 이베스트투자증권 어규진 연구원은  2분기에 메모리 가격 상승 및 OLED 패널 판매 호조가 지속되며 반도체와 디스플레이의 부품 사업부 실적 호조세가 지속될 전망 이라며  휴대폰 부분은 갤럭시S8 판매 호조에 따른 평균 판매가 상승 효과로 전작 대비 늦은 출시에도 견조한 실적을 유지할 전망 이라고 말했다.',
 ' 흥국증권 이민희 연구원은  삼성전자 2분기 영업이익 전망을 12조1000억원에서 13조원으로 상향 조정한다 며  메모리 가격 전망치 상향 조정과  노트7  리퍼폰 판매량을 일부 반영했다 고 설명했다.']

In [20]:
doc_info_list[0].keys()

dict_keys(['extract', 'extContent', 'rel_score', 'id', 'writetime'])